In [1]:
from bs4 import BeautifulSoup

In [2]:
import requests

In [3]:
import pandas as pd

In [4]:
url = "https://boston.craigslist.org/search/npo"

In [5]:
npo_jobs = {}
job_no = 0
while True:
    
    response = requests.get(url)
    data = response.text
    soup = BeautifulSoup(data, 'html.parser')
    jobs = soup.find_all("p", {"class":"result-info"})
    
    for job in jobs:
        
        title = job.find("a", {"class": "result-title"}).text
        location_tag = job.find("span", {"class": "result-hood"})
        location = location_tag.text[2:-1] if location_tag else "N/A"
        date = job.find("time", {"class": "result-date"}).text
        link = job.find("a", {"class": "result-title"}).get("href")
        job_response = requests.get(link)
        job_data = job_response.text
        job_soup = BeautifulSoup(job_data, "html.parser")
        job_description = job_soup.find("section", {"id":"postingbody"}).text
        job_attributes_tag = job_soup.find("p", {"class": "attrgroup"})
        job_attributes = job_attributes_tag.text if location_tag else "N/A"
        
        job_no+=1
        npo_jobs[job_no] = {title, location, date, link, job_attributes, job_description}
        
        #print("job Title: ", title, "\nLocation", location, "\nDate", date, "\nLink", link, "\nJob attributes", job_attributes, "\nJob Description", job_description, "\n---")
        
    url_tag = soup.find("a", {"title":"next page"})
    if url_tag.get("href"):
        url = "https://boston.craigslist.org/search/npo" + url_tag.get("href")
        print(url)
    else:
        break
            
print("total jobs: ", job_no)
npo_jobs_df = pd.DataFrame.from_dict(npo_jobs, orient = "index", columns = ["job title", "Location", "Date", "Link", "Job Attributes", "Job Description"])


https://boston.craigslist.org/search/npo/search/npo?s=120
https://boston.craigslist.org/search/npo/search/npo?s=240
total jobs:  242


In [6]:
npo_jobs_df.head()

,job title,Location,Date,Link,Job Attributes,Job Description
1,https://boston.craigslist.org/gbs/npo/d/organi...,*ORGANIZE YOUR COMMUNITY. STOP ATTACKS ON HEAL...,Jun 10,\n\nQR Code Link to This Post\n\n\n\n\n\n\nApp...,South Station,\ncompensation: $12-$16/hour\n\nemployment typ...
2,\n\nQR Code Link to This Post\n\n\nCLICK THE P...,Jun 10,Harvard Square,\ncompensation: $12-$16/hr\n\nemployment type:...,https://boston.craigslist.org/gbs/npo/d/cambri...,Find a career that matters🌏HELP CHANGE THE WOR...
3,Jun 10,PART TIME MOBILITY,"BOSTON, MA",https://boston.craigslist.org/bmw/npo/d/boston...,\n\nQR Code Link to This Post\n\n\nParatransit...,\ncompensation: $18.00-$20.00 per hour. Posit...
4,Boston,**SUMMER JOBS** WORK THAT MATTERS-AVG $480-750/WK,Jun 10,https://boston.craigslist.org/gbs/npo/d/boston...,\ncompensation: 12-17$/hour\n\nemployment type...,\n\nQR Code Link to This Post\n\n\n\n\n\n\n\n\n\n
5,N/A,Jun 10,Development Director,\n\nQR Code Link to This Post\n\n\nDEVELOPMENT...,https://boston.craigslist.org/nos/npo/d/salem-...,None


In [7]:
npo_jobs_df.to_csv("npo_jobs_details.csv")